## Taller N°1 : Aplicacion de Algoritmos de B´usqueda

* Profesor: Jorge Ivan Padilla Buritica
* Estudiante: Juan Felipe Cardona Arango 
* Maestria en Ciencia de Datos y Analitica
* Curso Fundamentos de Inteligencia Artificial
* 2025-02


## Preparacion del entorno de trabajo

In [1]:
import math
import heapq
import time
from time import time

# Punto 1:Optimizacion de rutas rurales (Logistica)

## Pregunta orientadora
**¿Cómo llego del nodo de inicio al nodo final gastando lo menos posible?**

---

## Algoritmos usados
- **UCS (Búsqueda de Costo Uniforme)**: explora rutas siempre expandiendo el camino más barato encontrado hasta ahora. Garantiza el camino óptimo si los pesos son positivos.
- **A***: es parecido a UCS, pero usa una **heurística** que le da una idea de qué tan cerca está del destino. Esto le permite ser más rápido sin perder la exactitud si la heurística está bien diseñada.

---

## ¿Qué es la heurística?
Una heurística es una **estimación** del costo que falta para llegar al destino. Por ejemplo, si conocemos las coordenadas de cada punto, podemos usar la **distancia en línea recta** (distancia euclidiana) como estimación.

> Si la línea recta siempre es igual o menor que cualquier ruta real, no se sobreestima el costo y A* seguirá encontrando la mejor ruta.

---

## Solución propuesta
1. Representar el mapa como un grafo con nodos, conexiones y pesos.
2. Elegir el nodo inicial y el nodo destino.
3. Seleccionar el algoritmo (UCS o A*) y, si es A*, definir la heurística.
4. Explorar el grafo paso a paso, comparando costos acumulados.
5. Cuando se llega al destino, devolver el camino encontrado y su costo total.

---

## Medición
Para evaluar un algoritmo no basta con que encuentre el camino óptimo; también importa **el esfuerzo que le costó encontrarlo**.

Por eso medimos:
- **Costo total** del camino encontrado.
- **Nodos expandidos** durante la búsqueda.
- **Profundidad** de la solución (número de pasos o aristas).
- **Tiempo de ejecución** (en milisegundos).



## Pseudocodigo

### UCS

Entrada: grafo G, inicio s, objetivo t, función de costo w(u,v) > 0
Salida: camino óptimo s→t (si existe) y su costo g*(t)

1  open ← PriorityQueue()                      // Cola por costo acumulado g(n)
2  open.push(s, key=0)
3  g[s] ← 0                                     // Costo al inicio es 0
4  parent[s] ← None                             // Para reconstruir el camino
5  depth[s] ← 0
6  visited ← ∅                                  // Conjunto de nodos ya extraídos (cerrados)
7  expanded ← 0                                  // Métrica: conteo de expansiones
8  while open ≠ ∅:
9      (n, _) ← open.pop_min()                  // Extraer nodo con menor g(n)
10     if n ∈ visited:                          // Si ya fue cerrado, saltar
11         continue
12     visited ← visited ∪ {n}
13     expanded ← expanded + 1
14     if n == t:                                // ¡Objetivo alcanzado!
15         return Reconstruir(parent, t), g[t], expanded, depth[t]
16     for cada m ∈ neighbors(n):                // Explorar vecinos
17         costo_tentativo ← g[n] + w(n,m)       // Relajar arista (n,m)
18         if m ∉ g or costo_tentativo < g[m]:   // ¿mejor camino a m?
19             g[m] ← costo_tentativo
20             parent[m] ← n
21             depth[m] ← depth[n] + 1
22             open.push_or_decrease(m, key=g[m])
23  return ∅, ∞, expanded, -1  


### A* 

Entrada: grafo G, inicio s, objetivo t, costo w(u,v) > 0, heurística h(n)
Salida: camino óptimo s→t (si h es admisible y consistente) y su costo g*(t)

1  open ← PriorityQueue()                       // Cola por f(n)=g(n)+h(n)
2  open.push(s, key=h(s))
3  g[s] ← 0
4  parent[s] ← None
5  depth[s] ← 0
6  closed ← ∅                                    // Nodos ya expandidos (cerrados)
7  expanded ← 0
8  while open ≠ ∅:
9      (n, _) ← open.pop_min()                   // Extraer el de menor f
10     if n ∈ closed:                            // Evitar re-procesar cerrados
11         continue
12     closed ← closed ∪ {n}
13     expanded ← expanded + 1
14     if n == t:                                 // Objetivo alcanzado
15         return Reconstruir(parent, t), g[t], expanded, depth[t]
16     for cada m ∈ neighbors(n):                 // Explorar vecinos
17         costo_tentativo ← g[n] + w(n,m)        // Relajar (n,m)
18         if m ∉ g or costo_tentativo < g[m]:    // ¿mejor g para m?
19             g[m] ← costo_tentativo
20             parent[m] ← n
21             depth[m] ← depth[n] + 1
22             f_m ← g[m] + h(m)                  // Clave f para la cola
23             open.push_or_decrease(m, key=f_m)
24  return ∅, ∞, expanded, -1

## Codigo

## Implementacion

In [2]:
from __future__ import annotations
from typing import Dict, Tuple, Callable, List, Any
import math
import time
import heapq
import networkx as nx


In [3]:
Position = Tuple[float, float]
Heuristic = Callable[[Any], float]

class PriorityQueue:
    def __init__(self):
        self._pq = []
        self._entry_finder = {}
        self._REMOVED = object()
        self._counter = 0
    def push_or_decrease(self, item, priority: float):
        if item in self._entry_finder:
            # marca entrada anterior como removida
            old = self._entry_finder.pop(item)
            old[-1] = self._REMOVED
        entry = [priority, self._counter, item]
        self._counter += 1
        self._entry_finder[item] = entry
        heapq.heappush(self._pq, entry)
    def pop_min(self):
        while self._pq:
            priority, _, item = heapq.heappop(self._pq)
            if item is not self._REMOVED:
                self._entry_finder.pop(item, None)
                return item, priority
        raise KeyError('pop from an empty priority queue')
    def empty(self):
        return not self._entry_finder

class SearchResult:
    def __init__(self, path: List[Any], cost: float, expanded: int, depth: int, runtime_ms: float):
        self.path = path
        self.cost = cost
        self.expanded = expanded
        self.depth = depth
        self.runtime_ms = runtime_ms
    def as_dict(self):
        return {
            'path': self.path,
            'cost': self.cost,
            'expanded': self.expanded,
            'depth': self.depth,
            'runtime_ms': round(self.runtime_ms, 3)
        }

# -----------------
# Heurísticas
# -----------------

def h_zero(goal: Any) -> Heuristic:
    return lambda n: 0.0

def h_euclid(G: nx.Graph, goal: Any) -> Heuristic:
    xg, yg = G.nodes[goal]['pos']
    def _h(n):
        x, y = G.nodes[n]['pos']
        return math.hypot(x - xg, y - yg)
    return _h

# Escalada controlada (para estudio de sobreestimación)
# h'(n) = alpha * h_euclid(n), con alpha >= 1

def h_scaled(G: nx.Graph, goal: Any, alpha: float = 1.2) -> Heuristic:
    base = h_euclid(G, goal)
    return lambda n: alpha * base(n)

# -----------------
# Reconstrucción de camino
# -----------------

def reconstruct(parent: Dict[Any, Any], goal: Any):
    path = []
    n = goal
    while n is not None:
        path.append(n)
        n = parent.get(n, None)
    path.reverse()
    return path

# -----------------
# UCS
# -----------------

def ucs(G: nx.Graph, start: Any, goal: Any) -> SearchResult:
    t0 = time.perf_counter()
    openq = PriorityQueue()
    openq.push_or_decrease(start, 0.0)
    g = {start: 0.0}
    parent = {start: None}
    expanded = 0
    visited = set()
    depth = {start: 0}
    while not openq.empty():
        n, _ = openq.pop_min()
        if n in visited:
            continue
        visited.add(n)
        expanded += 1
        if n == goal:
            t1 = time.perf_counter()
            path = reconstruct(parent, goal)
            return SearchResult(path, g[goal], expanded, depth[goal], (t1 - t0)*1000)
        for nbr in G.neighbors(n):
            w = G[n][nbr]['weight']
            tentative = g[n] + w
            if nbr not in g or tentative < g[nbr]:
                g[nbr] = tentative
                parent[nbr] = n
                depth[nbr] = depth[n] + 1
                openq.push_or_decrease(nbr, g[nbr])
    return SearchResult([], float('inf'), expanded, -1, (time.perf_counter() - t0)*1000)

# -----------------
# A*
# -----------------

def astar(G: nx.Graph, start: Any, goal: Any, h: Heuristic) -> SearchResult:
    t0 = time.perf_counter()
    openq = PriorityQueue()
    openq.push_or_decrease(start, h(start))
    g = {start: 0.0}
    parent = {start: None}
    depth = {start: 0}
    closed = set()
    expanded = 0
    while not openq.empty():
        n, _ = openq.pop_min()
        if n in closed:
            continue
        closed.add(n)
        expanded += 1
        if n == goal:
            t1 = time.perf_counter()
            path = reconstruct(parent, goal)
            return SearchResult(path, g[goal], expanded, depth[goal], (t1 - t0)*1000)
        for nbr in G.neighbors(n):
            w = G[n][nbr]['weight']
            tentative = g[n] + w
            if nbr not in g or tentative < g[nbr]:
                g[nbr] = tentative
                parent[nbr] = n
                depth[nbr] = depth[n] + 1
                f = g[nbr] + h(nbr)
                openq.push_or_decrease(nbr, f)
    return SearchResult([], float('inf'), expanded, -1, (time.perf_counter() - t0)*1000)

# -----------------
# Construcción de grafo de ejemplo
# -----------------

def build_sample_graph() -> Tuple[nx.Graph, Any, Any]:
    G = nx.Graph()
    # nodos con posiciones (x,y)
    coords: Dict[str, Position] = {
        'A': (0, 0), 'B': (2, 1), 'C': (4, 0),
        'D': (1, 3), 'E': (3, 3), 'F': (5, 2)
    }
    for n, pos in coords.items():
        G.add_node(n, pos=pos)
    def euclid(u,v):
        x1,y1 = G.nodes[u]['pos']
        x2,y2 = G.nodes[v]['pos']
        return math.hypot(x1-x2, y1-y2)
    # aristas (puedes ajustar para simular cierres/restricciones)
    edges = [
        ('A','B'), ('B','C'), ('A','D'), ('B','D'),
        ('B','E'), ('C','F'), ('D','E'), ('E','F')
    ]
    for u,v in edges:
        G.add_edge(u, v, weight=euclid(u,v))
    return G, 'A', 'F'

# -----------------
# Runner de experimentos (automatización comparativa)
# -----------------

def run_compare(G: nx.Graph, start: Any, goal: Any, heuristic_name: str = 'euclid', alpha: float = 1.2):
    results = []
    # UCS (baseline)
    r_ucs = ucs(G, start, goal)
    results.append(('UCS', 'h=0', r_ucs))
    # A* con h=0 (equivalente a UCS)
    r_h0 = astar(G, start, goal, h_zero(goal))
    results.append(('A*', 'h=0', r_h0))
    # A* euclidiana
    if heuristic_name == 'euclid':
        r_astar = astar(G, start, goal, h_euclid(G, goal))
        results.append(('A*', 'h=euclid', r_astar))
    # A* con sobreestimación controlada (para análisis)
    r_scaled = astar(G, start, goal, h_scaled(G, goal, alpha=alpha))
    results.append(('A*', f'h={alpha}*euclid', r_scaled))
    # Formato salida
    table = []
    for algo, hname, r in results:
        table.append({
            'algo': algo,
            'heuristic': hname,
            **r.as_dict()
        })
    return table


In [4]:
if __name__ == '__main__':
    G, s, t = build_sample_graph()
    table = run_compare(G, s, t, heuristic_name='euclid', alpha=1.3)
    for row in table:
        print(row)

{'algo': 'UCS', 'heuristic': 'h=0', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 6, 'depth': 3, 'runtime_ms': 0.15}
{'algo': 'A*', 'heuristic': 'h=0', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 6, 'depth': 3, 'runtime_ms': 0.022}
{'algo': 'A*', 'heuristic': 'h=euclid', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 5, 'depth': 3, 'runtime_ms': 0.019}
{'algo': 'A*', 'heuristic': 'h=1.3*euclid', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 4, 'depth': 3, 'runtime_ms': 0.015}


## Interpretación de resultados y comparación de algoritmos
A continuación se comentan las métricas obtenidas en el ejemplo para cada algoritmo:

## Resultados obtenidos
| Algoritmo | Heurística       | Camino encontrado       | Costo   | Nodos expandidos | Profundidad | Tiempo (ms) |
|-----------|------------------|------------------------|---------|------------------|-------------|-------------|
| UCS       | h=0              | A → B → C → F           | 6.7082  | 6                | 3           | 0.150       |
| A*        | h=0              | A → B → C → F           | 6.7082  | 6                | 3           | 0.022       |
| A*        | h=euclid         | A → B → C → F           | 6.7082  | 5                | 3           | 0.019       |
| A*        | h=1.3×euclid     | A → B → C → F           | 6.7082  | 4                | 3           | 0.015       |

---

**Interpretación:**
- Todas las configuraciones encontraron el mismo camino óptimo con costo ≈ 6.71.
- Usar una heurística informativa (euclidiana) redujo la cantidad de nodos expandidos y el tiempo de ejecución frente a UCS.
- La heurística escalada (1.3×euclid) aceleró aún más la búsqueda, aunque en grafos más complejos podría perder optimalidad.



## Respuestas a Preguntas Orientadoras

**1) ¿Cómo afecta la calidad de la heurística?**
> Una heurística más informativa (euclidiana) reduce el número de nodos expandidos y el tiempo de ejecución sin perder optimalidad. La heurística escalada (1.3×euclid) reduce aún más las expansiones, pero podría sacrificar optimalidad en casos más complejos.

**2) ¿Es pertinente la euclidiana?**
> Sí, porque el peso de las aristas se calculó como distancia euclidiana. Esto garantiza que la heurística sea admisible y consistente.

**3) ¿Flexibilidad ante nuevas rutas o restricciones?**
> El modelo permite añadir o quitar aristas y ajustar pesos fácilmente. Los algoritmos pueden recalcular rutas óptimas sin modificar la lógica interna.

**4) ¿Escalabilidad si el grafo crece 10× en tamaño?**
> UCS y A* seguirán funcionando, pero el tiempo y memoria crecerán. Una heurística de calidad (como la euclidiana) será clave para reducir expansiones. En grafos muy grandes se recomienda optimizar con técnicas como búsqueda bidireccional o preprocesamiento de rutas.

---

# Punto 2: Red de metro (transporte)
Se proporciona un grafo no ponderado de estaciones de metro. Compare las soluciones usando **BFS** (Breadth-First Search) e **IDS** (Iterative Deepening Search) para ir de la estación A a la estación J.

**Cuestiones críticas**:
- ¿Qué pasa si el sistema crece a 100 nodos? ¿Cuál es el impacto en complejidad temporal?
- ¿Puede automatizarse la elección del mejor algoritmo según el tamaño de la red?

Cada ejercicio debe incluir:
- Análisis del problema y del tipo de búsqueda.
- Implementación en Python y explicación del código.
- Medidas de desempeño (tiempo, nodos expandidos, profundidad y costo).
- Evaluación de complejidad temporal y espacial.
- Rutinas automáticas de comparación entre algoritmos.
- Reflexión crítica sobre eficiencia y aplicabilidad.


## Pseudocodigo 

In [ ]:
## Codigo 

## Pseudocodigo

Entrada: G=(V,E), inicio s, destino t, weight(u,v) en minutos, h(n)
Salida: ruta óptima y tiempo total

1  open ← PriorityQueue()
2  open.push(s, key=h(s))
3  g[s] ← 0
4  parent[s] ← None
5  closed ← ∅
6  expanded ← 0
7  while open ≠ ∅:
8      (n, _) ← open.pop_min()
9      if n ∈ closed: continue
10     closed ← closed ∪ {n}
11     expanded ← expanded + 1
12     if n == t:
13         return Reconstruir(parent, t), g[t], expanded
14     for cada m ∈ vecinos(n):
15         costo_tent ← g[n] + weight(n,m)
16         if m ∉ g or costo_tent < g[m]:
17             g[m] ← costo_tent
18             parent[m] ← n
19             f_m ← g[m] + h(m)
20             open.push_or_decrease(m, f_m)
21 return ∅, ∞, expanded




In [ ]:
## Codigo

In [48]:
# -*- coding: utf-8 -*-
"""
Comparativa de algoritmos de búsqueda en una red de metro (A*, BFS e IDS)
--------------------------------------------------------------------------

Este script implementa y compara tres enfoques clásicos de búsqueda:
  - A*: óptimo en COSTO (tiempo) con heurística admisible.
  - BFS (Breadth-First Search): óptimo en NÚMERO DE ARISTAS (hops) cuando los pesos son iguales.
  - IDS (Iterative Deepening Search): óptimo en NÚMERO DE ARISTAS con baja memoria, pero más re-expansiones.

Incluye:
  1) Construcción de dos grafos de prueba:
     - Pequeño (A..J) con pesos heterogéneos y atajos (bypasses).
     - Grande (~100 nodos) como grilla 10x10 con pesos aleatorios.
  2) Implementaciones propias de A*, BFS e IDS (sin depender de los equivalentes de NetworkX).
  3) Métricas de desempeño: tiempo, nodos expandidos, profundidad (hops) y costo total (minutos).
  4) Heurística admisible para A*: distancia en línea recta / v_max.
  5) Selector automático de algoritmo por reglas sencillas.
"""

import time
import math
import random
from collections import deque
from typing import Dict, List, Optional, Tuple

import networkx as nx
import pandas as pd


# ---------------------------------------------------------------------
# Utilidades
# ---------------------------------------------------------------------

def reconstruct(parent: Dict[str, Optional[str]], goal: str) -> List[str]:
    """Reconstruye la ruta desde el 'start' implícito hasta el 'goal'."""
    path = []
    n = goal
    while n is not None:
        path.append(n)
        n = parent.get(n)
    return list(reversed(path))


def path_cost(G: nx.Graph, path: List[str]) -> float:
    """Calcula el costo total (minutos) de una ruta dada."""
    if not path or len(path) == 1:
        return 0.0
    total = 0.0
    for u, v in zip(path[:-1], path[1:]):
        total += G[u][v].get('weight', 1.0)
    return total


# ---------------------------------------------------------------------
# Construcción de grafos
# ---------------------------------------------------------------------

def build_metro_graph_small() -> Tuple[nx.Graph, str, str]:
    """Crea un grafo pequeño de prueba (A..J + rama E1-F1)."""
    G = nx.Graph()
    coords = {
        'A': (0, 0), 'B': (2, 0), 'C': (4, 0), 'D': (6, 0),
        'E': (8, 0), 'F': (10, 0), 'G': (12, 0), 'H': (14, 0),
        'I': (16, 0), 'J': (18, 0),
        'E1': (8, -2), 'F1': (10, -2),
    }
    for s, pos in coords.items():
        G.add_node(s, pos=pos)
    edges = [
        ('A','B',3), ('B','C',6), ('C','D',3), ('D','E',5),
        ('E','F',4), ('F','G',6), ('G','H',3), ('H','I',4), ('I','J',5),
        ('B','D',4), ('E','G',5), ('G','I',6),
        ('E','E1',2), ('E1','F1',3), ('F1','F',2)
    ]
    for u, v, w in edges:
        G.add_edge(u, v, weight=w)
    return G, 'A', 'J'


def build_grid_graph(n_rows: int = 10, n_cols: int = 10, weight_range=(2, 8)) -> Tuple[nx.Graph, str, str]:
    """Crea un grafo tipo grilla con pesos aleatorios."""
    G = nx.Graph()
    for r in range(n_rows):
        for c in range(n_cols):
            name = f"r{r}c{c}"
            G.add_node(name, pos=(c*2, r*2))
    for r in range(n_rows):
        for c in range(n_cols):
            u = f"r{r}c{c}"
            if r + 1 < n_rows:
                v = f"r{r+1}c{c}"
                G.add_edge(u, v, weight=random.randint(*weight_range))
            if c + 1 < n_cols:
                v = f"r{r}c{c+1}"
                G.add_edge(u, v, weight=random.randint(*weight_range))
    return G, "r0c0", f"r{n_rows-1}c{n_cols-1}"


# ---------------------------------------------------------------------
# Heurística para A*
# ---------------------------------------------------------------------

def h_metro(G: nx.Graph, goal: str, vmax: float = 0.667):
    """Heurística: distancia euclidiana / velocidad máxima."""
    xg, yg = G.nodes[goal]['pos']
    def h(n: str) -> float:
        x, y = G.nodes[n]['pos']
        dist = math.hypot(x - xg, y - yg)
        return dist / vmax
    return h


# ---------------------------------------------------------------------
# Algoritmos de búsqueda
# ---------------------------------------------------------------------

import heapq

def astar_metro(G: nx.Graph, start: str, goal: str, hfunc):
    """Implementación de A*."""
    openq = []
    heapq.heappush(openq, (hfunc(start), start))
    g = {start: 0.0}
    parent = {start: None}
    expanded = 0
    closed = set()
    while openq:
        _, n = heapq.heappop(openq)
        if n in closed:
            continue
        closed.add(n)
        expanded += 1
        if n == goal:
            path = reconstruct(parent, goal)
            return path, path_cost(G, path), expanded, len(path)-1
        for nbr in G.neighbors(n):
            w = G[n][nbr].get('weight', 1.0)
            tentative = g[n] + w
            if tentative < g.get(nbr, float('inf')):
                g[nbr] = tentative
                parent[nbr] = n
                heapq.heappush(openq, (tentative + hfunc(nbr), nbr))
    return [], float('inf'), expanded, None


def bfs_unweighted(G: nx.Graph, start: str, goal: str):
    """Búsqueda en anchura (óptima en hops si pesos iguales)."""
    q = deque([start])
    parent = {start: None}
    visited = {start}
    expanded = 0
    while q:
        n = q.popleft()
        expanded += 1
        if n == goal:
            path = reconstruct(parent, goal)
            return path, path_cost(G, path), expanded, len(path)-1
        for nbr in G.neighbors(n):
            if nbr not in visited:
                visited.add(nbr)
                parent[nbr] = n
                q.append(nbr)
    return [], float('inf'), expanded, None


def dls(G, current, goal, limit, parent, visited, expanded):
    """Búsqueda en profundidad limitada."""
    expanded[0] += 1
    if current == goal:
        return True
    if limit == 0:
        return False
    for nbr in G.neighbors(current):
        if nbr not in visited:
            visited.add(nbr)
            parent[nbr] = current
            if dls(G, nbr, goal, limit-1, parent, visited, expanded):
                return True
    return False


def ids_unweighted(G, start, goal, max_depth=1000):
    """Búsqueda con profundización iterativa."""
    expanded_total = 0
    for depth in range(max_depth+1):
        parent = {start: None}
        visited = {start}
        expanded = [0]
        found = dls(G, start, goal, depth, parent, visited, expanded)
        expanded_total += expanded[0]
        if found:
            path = reconstruct(parent, goal)
            return path, path_cost(G, path), expanded_total, len(path)-1, depth
    return [], float('inf'), expanded_total, None, None


# ---------------------------------------------------------------------
# Ejecución y comparación
# ---------------------------------------------------------------------

def run_experiments():
    # --- Grafo pequeño ---
    Gs, s, t = build_metro_graph_small()
    hs = h_metro(Gs, t)
    t0 = time.perf_counter()
    pa, ca, ea, da = astar_metro(Gs, s, t, hs)
    t1 = time.perf_counter()
    t2 = time.perf_counter()
    pb, cb, eb, db = bfs_unweighted(Gs, s, t)
    t3 = time.perf_counter()
    t4 = time.perf_counter()
    pi, ci, ei, di, limi = ids_unweighted(Gs, s, t, max_depth=50)
    t5 = time.perf_counter()
    df_small = pd.DataFrame([
        {"Algoritmo": "A*", "Ruta": "→".join(pa), "Costo (min)": round(ca,2),
         "Profundidad": da, "Nodos expandidos": ea, "Tiempo (ms)": round((t1-t0)*1000,3)},
        {"Algoritmo": "BFS", "Ruta": "→".join(pb), "Costo (min)": round(cb,2),
         "Profundidad": db, "Nodos expandidos": eb, "Tiempo (ms)": round((t3-t2)*1000,3)},
        {"Algoritmo": "IDS", "Ruta": "→".join(pi), "Costo (min)": round(ci,2),
         "Profundidad": di, "Nodos expandidos": ei, "Tiempo (ms)": round((t5-t4)*1000,3)}
    ])
    print("\n=== Resultados - Grafo A..J ===")
    print(df_small.to_string(index=False))

    # --- Grafo grande ---
    Gg, sg, tg = build_grid_graph(10, 10, (2,8))
    hg = h_metro(Gg, tg)
    tb0 = time.perf_counter()
    pa_b, ca_b, ea_b, da_b = astar_metro(Gg, sg, tg, hg)
    tb1 = time.perf_counter()
    tb2 = time.perf_counter()
    pb_b, cb_b, eb_b, db_b = bfs_unweighted(Gg, sg, tg)
    tb3 = time.perf_counter()
    tb4 = time.perf_counter()
    pi_b, ci_b, ei_b, di_b, limi_b = ids_unweighted(Gg, sg, tg, max_depth=200)
    tb5 = time.perf_counter()
    df_big = pd.DataFrame([
        {"Algoritmo": "A*", "Long. ruta": len(pa_b), "Costo (min)": round(ca_b,2),
         "Profundidad": da_b, "Nodos expandidos": ea_b, "Tiempo (ms)": round((tb1-tb0)*1000,3)},
        {"Algoritmo": "BFS", "Long. ruta": len(pb_b), "Costo (min)": round(cb_b,2),
         "Profundidad": db_b, "Nodos expandidos": eb_b, "Tiempo (ms)": round((tb3-tb2)*1000,3)},
        {"Algoritmo": "IDS", "Long. ruta": len(pi_b), "Costo (min)": round(ci_b,2),
         "Profundidad": di_b, "Nodos expandidos": ei_b, "Tiempo (ms)": round((tb5-tb4)*1000,3)}
    ])
    print("\n=== Resultados - Grafo 10x10 (~100 nodos) ===")
    print(df_big.to_string(index=False))


if __name__ == "__main__":
    run_experiments()



=== Resultados - Grafo A..J ===
Algoritmo                Ruta  Costo (min)  Profundidad  Nodos expandidos  Tiempo (ms)
       A*       A→B→D→E→G→I→J         28.0            6                 9        0.058
      BFS       A→B→D→E→G→I→J         28.0            6                12        0.015
      IDS A→B→C→D→E→F→G→H→I→J         39.0            9                65        0.030

=== Resultados - Grafo 10x10 (~100 nodos) ===
Algoritmo  Long. ruta  Costo (min)  Profundidad  Nodos expandidos  Tiempo (ms)
       A*          19         57.0           18                56        0.300
      BFS          19         90.0           18               100        0.063
      IDS          41        209.0           40              2072        0.803


## Respuestas a Preguntas Orientadoras

# Punto 3 :Filogenia biologia

Dada una red de metro representada como un grafo ponderado (nodos = estaciones, aristas = tramos entre estaciones), encontrar la ruta de menor tiempo entre dos estaciones usando el algoritmo A* con una heurística apropiada.

Se requiere encontrar la ruta de menor tiempo entre dos estaciones de metro en un grafo ponderado, considerando:

Tramos con tiempos variables.

Posibles penalizaciones por transferencia.

Posibles cierres de tramos.

El problema es una instancia del camino mínimo en grafos ponderados con pesos positivos, para lo cual se emplea A* con una heurística admisible que acelera la búsqueda respecto a UCS.


## Pseudocodigo

## Codigo

In [49]:
# -*- coding: utf-8 -*-
"""
MRCA/LCA con Búsqueda en Profundidad Limitada (DLS) y comparadores.
- Árbol enraizado: parent -> child (dirigido, acíclico).
- MRCA por "prefijo común más profundo" de las rutas raíz->u y raíz->v.
- Incluye:
  * DLS + IDS para obtener rutas (cumple consigna).
  * BFS desde raíz como baseline.
  * Métricas: tiempo, nodos expandidos, profundidad de rutas.
  * Rutina de comparación automática entre DLS y BFS (y placeholder para Euler Tour).
"""

from typing import Dict, List, Optional, Tuple
import time
from collections import deque

# ----------------------------
# Estructura del árbol
# ----------------------------
class PhyloTree:
    def __init__(self, root: str):
        self.root = root
        self.children: Dict[str, List[str]] = {}
        self.parent: Dict[str, Optional[str]] = {root: None}

    def add_edge(self, p: str, c: str):
        self.children.setdefault(p, []).append(c)
        self.children.setdefault(c, [])
        self.parent[c] = p
        self.parent.setdefault(p, None)

# ----------------------------
# DLS para ruta raíz->target
# ----------------------------
def dls_path(tree: PhyloTree, current: str, target: str, limit: int,
             path: List[str], out_path: List[str], counters: Dict[str, int]) -> bool:
    counters["expanded"] += 1
    path.append(current)
    if current == target:
        out_path[:] = path[:]
        path.pop()
        return True
    if limit == 0:
        path.pop()
        return False
    for ch in tree.children.get(current, []):
        if dls_path(tree, ch, target, limit-1, path, out_path, counters):
            path.pop()
            return True
    path.pop()
    return False

def path_by_ids(tree: PhyloTree, target: str, max_depth: int, counters: Dict[str, int]) -> Optional[List[str]]:
    """
    IDS: profundidades crecientes 0..max_depth. Devuelve ruta raíz->target.
    'counters["expanded"]' acumula nodos expandidos.
    """
    for L in range(max_depth + 1):
        tmp = []
        if dls_path(tree, tree.root, target, L, [], tmp, counters):
            return tmp
    return None

# ----------------------------
# BFS para ruta raíz->target
# ----------------------------
def path_by_bfs(tree: PhyloTree, target: str, counters: Dict[str, int]) -> Optional[List[str]]:
    q = deque([tree.root])
    parent: Dict[str, Optional[str]] = {tree.root: None}
    visited = {tree.root}
    while q:
        n = q.popleft()
        counters["expanded"] += 1
        if n == target:
            # reconstrucción
            path = []
            cur = n
            while cur is not None:
                path.append(cur)
                cur = parent[cur]
            return list(reversed(path))
        for ch in tree.children.get(n, []):
            if ch not in visited:
                visited.add(ch)
                parent[ch] = n
                q.append(ch)
    return None

# ----------------------------
# MRCA por prefijo común
# ----------------------------
def mrca_from_paths(path_u: List[str], path_v: List[str]) -> Optional[str]:
    mrca = None
    for a, b in zip(path_u, path_v):
        if a == b:
            mrca = a
        else:
            break
    return mrca

# ----------------------------
# Métrica de "costo" y profundidad
# ----------------------------
def path_depth(path: Optional[List[str]]) -> Optional[int]:
    return None if path is None else len(path) - 1

def path_cost_by_branch_length(path: Optional[List[str]], branch_len: Dict[Tuple[str,str], float]) -> Optional[float]:
    """
    Si existieran longitudes de rama (tiempo evolutivo), sumarlas como "costo".
    En este ejemplo, no usamos longitudes (sección 3 justifica la métrica),
    pero dejamos la rutina para escenarios reales.
    """
    if path is None or len(path) < 2:
        return 0.0 if path is not None else None
    total = 0.0
    for u, v in zip(path[:-1], path[1:]):
        total += branch_len.get((u,v), 1.0)  # por defecto 1.0 si no hay info
    return total

# ----------------------------
# Orquestación: MRCA con DLS o BFS
# ----------------------------
def lca_with_method(tree: PhyloTree, u: str, v: str, method: str, max_depth: int = 1000):
    counters_u = {"expanded": 0}
    counters_v = {"expanded": 0}
    t0 = time.perf_counter()

    if method == "DLS":
        pu = path_by_ids(tree, u, max_depth, counters_u)
        pv = path_by_ids(tree, v, max_depth, counters_v)
    elif method == "BFS":
        pu = path_by_bfs(tree, u, counters_u)
        pv = path_by_bfs(tree, v, counters_v)
    else:
        raise ValueError("method debe ser 'DLS' o 'BFS'")

    t1 = time.perf_counter()
    lca = mrca_from_paths(pu, pv) if pu and pv else None

    metrics = {
        "tiempo_ms": round((t1 - t0) * 1000, 3),
        "expandidos_u": counters_u["expanded"],
        "expandidos_v": counters_v["expanded"],
        "profundidad_u": path_depth(pu),
        "profundidad_v": path_depth(pv),
        "ruta_u": pu,
        "ruta_v": pv,
        "LCA": lca
    }
    return metrics

# ----------------------------
# Demo mínima
# ----------------------------
if __name__ == "__main__":
    # Árbol de ejemplo:
    #         Root
    #        /    \
    #    Clade1  Clade2
    #     /  \     /  \
    #   SppA SppB SppC SppD
    T = PhyloTree("Root")
    T.add_edge("Root", "Clade1"); T.add_edge("Root", "Clade2")
    T.add_edge("Clade1", "SppA"); T.add_edge("Clade1", "SppB")
    T.add_edge("Clade2", "SppC"); T.add_edge("Clade2", "SppD")

    pares = [("SppA", "SppB"), ("SppA", "SppC"), ("SppC", "SppD")]

    for u, v in pares:
        print(f"\nPar {u}-{v} (DLS):")
        print(lca_with_method(T, u, v, method="DLS", max_depth=3))
        print(f"Par {u}-{v} (BFS):")
        print(lca_with_method(T, u, v, method="BFS"))



Par SppA-SppB (DLS):
{'tiempo_ms': 0.026, 'expandidos_u': 7, 'expandidos_v': 8, 'profundidad_u': 2, 'profundidad_v': 2, 'ruta_u': ['Root', 'Clade1', 'SppA'], 'ruta_v': ['Root', 'Clade1', 'SppB'], 'LCA': 'Clade1'}
Par SppA-SppB (BFS):
{'tiempo_ms': 0.028, 'expandidos_u': 4, 'expandidos_v': 5, 'profundidad_u': 2, 'profundidad_v': 2, 'ruta_u': ['Root', 'Clade1', 'SppA'], 'ruta_v': ['Root', 'Clade1', 'SppB'], 'LCA': 'Clade1'}

Par SppA-SppC (DLS):
{'tiempo_ms': 0.01, 'expandidos_u': 7, 'expandidos_v': 10, 'profundidad_u': 2, 'profundidad_v': 2, 'ruta_u': ['Root', 'Clade1', 'SppA'], 'ruta_v': ['Root', 'Clade2', 'SppC'], 'LCA': 'Root'}
Par SppA-SppC (BFS):
{'tiempo_ms': 0.011, 'expandidos_u': 4, 'expandidos_v': 6, 'profundidad_u': 2, 'profundidad_v': 2, 'ruta_u': ['Root', 'Clade1', 'SppA'], 'ruta_v': ['Root', 'Clade2', 'SppC'], 'LCA': 'Root'}

Par SppC-SppD (DLS):
{'tiempo_ms': 0.01, 'expandidos_u': 10, 'expandidos_v': 11, 'profundidad_u': 2, 'profundidad_v': 2, 'ruta_u': ['Root', 'Clade2',

## Respuestas a Preguntas Orientadoras

# Punto 4:Evaluacion de emergencia ( Infraestructura )

En un grafo ponderado donde el peso representa nivel de riesgo vial, encontrar con Búsqueda de Costo Uniforme (UCS) la ruta más segura (riesgo total mínimo) desde un punto de origen hacia una o varias salidas.



## Pseudocodigo

## Codigo

In [50]:
# -*- coding: utf-8 -*-
"""
Evacuación de emergencia: ruta más segura con UCS (Uniform Cost Search).
- Pesos = nivel de RIESGO (no distancia).
- Soporta múltiples salidas, cierres de tramos y escenarios con incertidumbre (Monte Carlo).
- Métricas: tiempo, nodos expandidos, profundidad (hops), riesgo acumulado.
"""

import heapq
import math
import random
import time
from collections import deque
from typing import Dict, List, Tuple, Optional, Set

# ---------------------------
# Grafo simple con API mínima
# ---------------------------
class Graph:
    def __init__(self):
        self.adj: Dict[str, List[Tuple[str, Dict]]] = {}

    def add_edge(self, u: str, v: str, risk: float,
                 closed: bool=False, mu: Optional[float]=None, sigma: Optional[float]=None):
        """
        risk: riesgo determinístico base (si no hay incertidumbre).
        mu, sigma: parámetros para riesgo estocástico (opcional).
        closed: si True, arista bloqueada.
        """
        self.adj.setdefault(u, []).append((v, {"risk": risk, "closed": closed, "mu": mu, "sigma": sigma}))
        self.adj.setdefault(v, []).append((u, {"risk": risk, "closed": closed, "mu": mu, "sigma": sigma}))

    def neighbors(self, u: str):
        return self.adj.get(u, [])

    def set_closed(self, u: str, v: str, closed: bool=True):
        """Cierra/abre una arista en ambas direcciones."""
        for i, (w, attrs) in enumerate(self.adj.get(u, [])):
            if w == v:
                attrs["closed"] = closed
        for i, (w, attrs) in enumerate(self.adj.get(v, [])):
            if w == u:
                attrs["closed"] = closed


# --------------------------------------
# Builders: Grafo pequeño y cuadrícula NxN
# --------------------------------------
def build_small_graph() -> Tuple[Graph, str, Set[str]]:
    G = Graph()
    # Nodos A..J (como ejemplo urbano simplificado)
    edges = [
        ("A","B", 3), ("B","C", 6), ("C","D", 3), ("D","E", 5),
        ("E","F", 4), ("F","G", 7), ("G","H", 3), ("H","I", 6), ("I","J", 4),
        # Atajos con diferente riesgo
        ("B","D", 2), ("E","G", 5), ("G","I", 5),
        # Ruta alternativa por zona más riesgosa
        ("C","F", 10), ("F","I", 9)
    ]
    for u,v,r in edges:
        G.add_edge(u,v,risk=r)
    start = "A"
    goals = {"J"}   # salida/s
    return G, start, goals

def build_grid_graph(n=10, rng=(1,10), p_block=0.05) -> Tuple[Graph, str, Set[str]]:
    """Cuadrícula n x n, riesgo aleatorio, algunos cierres simulados."""
    G = Graph()
    def name(r,c): return f"r{r}c{c}"
    for r in range(n):
        for c in range(n):
            if r+1<n:
                risk = random.randint(*rng)
                G.add_edge(name(r,c), name(r+1,c), risk=risk, closed=(random.random()<p_block))
            if c+1<n:
                risk = random.randint(*rng)
                G.add_edge(name(r,c), name(r,c+1), risk=risk, closed=(random.random()<p_block))
    start = "r0c0"
    goals = {f"r{n-1}c{n-1}"}  # esquina opuesta como salida
    return G, start, goals


# ---------------------------
# UCS (Uniform Cost Search)
# ---------------------------
def ucs_safest_path(G: Graph, start: str, goals: Set[str]) -> Tuple[List[str], float, int, int, float]:
    """
    Devuelve (ruta, riesgo_total, nodos_expandidos, profundidad_hops, tiempo_ms).
    Termina al extraer de la cola un objetivo (óptimo en riesgo).
    """
    t0 = time.perf_counter()
    openq = []
    heapq.heappush(openq, (0.0, start))
    parent: Dict[str, Optional[str]] = {start: None}
    best: Dict[str, float] = {start: 0.0}
    expanded = 0
    visited = set()

    while openq:
        g_u, u = heapq.heappop(openq)
        if u in visited:
            continue
        visited.add(u)
        expanded += 1
        if u in goals:
            path = []
            cur = u
            while cur is not None:
                path.append(cur)
                cur = parent[cur]
            path.reverse()
            t1 = time.perf_counter()
            return path, g_u, expanded, len(path)-1, round((t1-t0)*1000, 3)

        for v, attrs in G.neighbors(u):
            if attrs.get("closed", False):
                continue
            w = float(attrs.get("risk", 0.0))
            g_v = g_u + w
            if g_v < best.get(v, math.inf):
                best[v] = g_v
                parent[v] = u
                heapq.heappush(openq, (g_v, v))

    t1 = time.perf_counter()
    return [], math.inf, expanded, 0, round((t1-t0)*1000, 3)


# ---------------------------------------------------------
# (Opcional) Heurística admisible para A* basada en riesgos
# ---------------------------------------------------------
def a_star_safest_path(G: Graph, start: str, goals: Set[str]) -> Tuple[List[str], float, int, int, float]:
    """
    A* con heurística admisible:
    h(n) = min_risk_edge * min_hops_est(n,goal)
    Donde min_hops_est se aproxima con Manhattan en grillas o con BFS (sin pesos) para general.
    Si no puedes estimar hops, usar h=0 y A*==UCS.
    """
    # Estimar min_risk_edge global (lower-bound del riesgo por arista)
    min_risk = math.inf
    for u in G.adj:
        for v, attrs in G.adj[u]:
            if not attrs.get("closed", False):
                min_risk = min(min_risk, float(attrs.get("risk", math.inf)))
    if not math.isfinite(min_risk):
        min_risk = 0.0

    # Precompute un BFS no ponderado para hops minimos hacia algún goal (reverse-BFS)
    # Construimos grafo no ponderado sin edges cerradas
    rev_adj: Dict[str, List[str]] = {}
    for u in G.adj:
        for v, attrs in G.adj[u]:
            if not attrs.get("closed", False):
                rev_adj.setdefault(v, []).append(u)

    from collections import deque
    hop_dist = {g: 0 for g in goals}
    q = deque(goals)
    while q:
        x = q.popleft()
        for p in rev_adj.get(x, []):
            if p not in hop_dist:
                hop_dist[p] = hop_dist[x] + 1
                q.append(p)

    def h(n: str) -> float:
        hops = hop_dist.get(n, 0)  # 0 si desconocido (admisible)
        return hops * min_risk

    # A*
    t0 = time.perf_counter()
    openq = []
    g = {start: 0.0}
    parent = {start: None}
    heapq.heappush(openq, (h(start), start))
    expanded = 0
    closed = set()

    while openq:
        _, u = heapq.heappop(openq)
        if u in closed:
            continue
        closed.add(u)
        expanded += 1
        if u in goals:
            # reconstruir
            path = []
            cur = u
            while cur is not None:
                path.append(cur)
                cur = parent[cur]
            path.reverse()
            t1 = time.perf_counter()
            return path, g[u], expanded, len(path)-1, round((t1-t0)*1000, 3)

        for v, attrs in G.neighbors(u):
            if attrs.get("closed", False):
                continue
            w = float(attrs.get("risk", 0.0))
            tentative = g[u] + w
            if tentative < g.get(v, math.inf):
                g[v] = tentative
                parent[v] = u
                f = tentative + h(v)
                heapq.heappush(openq, (f, v))

    t1 = time.perf_counter()
    return [], math.inf, expanded, 0, round((t1-t0)*1000, 3)


# -----------------------------------------------------------
# (Opcional) Monte Carlo para riesgo estocástico por escenario
# -----------------------------------------------------------
def sample_edge_risk(attrs: Dict) -> float:
    """Muestra riesgo de N(mu, sigma) truncada a >= 0; fallback a 'risk' si no hay incertidumbre."""
    mu = attrs.get("mu", None)
    sigma = attrs.get("sigma", None)
    if mu is None or sigma is None:
        return float(attrs.get("risk", 0.0))
    # Muestra y trunca a 0
    x = random.gauss(mu, sigma)
    return max(0.0, x)

def ucs_under_scenarios(G: Graph, start: str, goals: Set[str], n_scenarios=200) -> Dict:
    """
    Ejecuta UCS sobre n escenarios de riesgo (mu/sigma) y resume:
    - riesgo esperado de la mejor ruta,
    - varianza,
    - probabilidad de superar umbral (CVaR proxy),
    - modo frecuente de ruta.
    """
    from collections import Counter
    risks, routes = [], []
    for _ in range(n_scenarios):
        # correr UCS con riesgos muestreados
        openq = []
        heapq.heappush(openq, (0.0, start))
        parent = {start: None}
        best = {start: 0.0}
        visited = set()
        while openq:
            g_u, u = heapq.heappop(openq)
            if u in visited:
                continue
            visited.add(u)
            if u in goals:
                # reconstruir
                path = []
                cur = u
                while cur is not None:
                    path.append(cur)
                    cur = parent[cur]
                path.reverse()
                risks.append(g_u)
                routes.append(tuple(path))
                break
            for v, attrs in G.neighbors(u):
                if attrs.get("closed", False):
                    continue
                w = sample_edge_risk(attrs)
                g_v = g_u + w
                if g_v < best.get(v, math.inf):
                    best[v] = g_v
                    parent[v] = u
                    heapq.heappush(openq, (g_v, v))

    import statistics as stats
    avg = stats.mean(risks) if risks else math.inf
    var = stats.pvariance(risks) if len(risks) > 1 else 0.0
    # Prob. de “alto riesgo” respecto a percentil 80 como umbral
    if risks:
        thr = sorted(risks)[int(0.8*len(risks))-1]
        p_hi = sum(r>thr for r in risks)/len(risks)
    else:
        thr = math.inf; p_hi = 0.0
    mode_route, freq = (None, 0)
    if routes:
        c = Counter(routes)
        mode_route, freq = c.most_common(1)[0]
    return {
        "n_scenarios": n_scenarios,
        "risk_avg": avg,
        "risk_var": var,
        "p_above_p80": p_hi,
        "p80_threshold": thr,
        "mode_route": list(mode_route) if mode_route else None,
        "mode_freq": freq/len(routes) if routes else 0.0
    }


# ---------------------------
# Experimentos rápidos
# ---------------------------
if __name__ == "__main__":
    random.seed(7)

    # Caso pequeño
    Gs, s, goals = build_small_graph()
    path, risk, expd, depth, tms = ucs_safest_path(Gs, s, goals)
    print("UCS pequeño:", path, risk, expd, depth, tms, "ms")

    # Comparación opcional con A*
    pathA, riskA, expdA, depthA, tmsA = a_star_safest_path(Gs, s, goals)
    print("A* pequeño:", pathA, riskA, expdA, depthA, tmsA, "ms")

    # Cuadrícula 10x10
    Gg, sg, ggoals = build_grid_graph(n=10, rng=(1,10), p_block=0.05)
    pathG, riskG, expdG, depthG, tmsG = ucs_safest_path(Gg, sg, ggoals)
    print("UCS 10x10:", len(pathG), riskG, expdG, depthG, tmsG, "ms")

    # Monte Carlo en pequeño (si quisieras, añade mu/sigma a algunas aristas y re-ejecuta)
    # results = ucs_under_scenarios(Gs, s, goals, n_scenarios=200)
    # print("MC resumen:", results)


UCS pequeño: ['A', 'B', 'D', 'E', 'G', 'I', 'J'] 24.0 10 6 0.021 ms
A* pequeño: ['A', 'B', 'D', 'E', 'G', 'I', 'J'] 24.0 10 6 0.015 ms
UCS 10x10: 19 58.0 99 18 0.127 ms


## Respuestas a Preguntas Orientadoras